In [1]:
from __modelUnpair__ import Encoder, Decoder, glob_all
from __modelUnpair__ import define_discriminator, define_generator, define_composite_model, train
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import *
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().disabled = True

import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
tf.config.optimizer.set_jit(True)

In [3]:
channel = 6; q = 1; resnet_n = 9; d_lr=5e-6; g_lr = 1e-3; epochs = 100; batch = 1
dataA1 = np.load('AIR73_Align_KNN_Training_del.npy', mmap_mode='r')
dataA2 = np.load('SAT73_Align_KNN_Training_del.npy', mmap_mode='r')

dataA = np.concatenate([dataA1,dataA2])
dataB= np.load('Unpair_train_Code'+str(channel)+'.npy', mmap_mode='r')


print('Loaded', dataA.shape, dataB.shape)

EPA73_test_pm25 = np.load('EPA73_testing_PM25.npy', mmap_mode='r')

station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')

ex5_KNN = np.load('EPA73_testing_KNN3fill_ex5.npy', mmap_mode='r')
ex10_KNN = np.load('EPA73_testing_KNN3fill_ex10.npy', mmap_mode='r')
ex15_KNN = np.load('EPA73_testing_KNN3fill_ex15.npy', mmap_mode='r')
ex20_KNN = np.load('EPA73_testing_KNN3fill_ex20.npy', mmap_mode='r')

model_path = 'TEC_result/'
if not os.path.exists(model_path):os.makedirs(model_path)

Loaded (3076, 348, 204, 1) (3076, 44, 26, 6)


In [4]:
x = Input(shape=(348, 204, 1))
Unpair_Autoencoder_weight = glob_all('TE_result/')[0]
Unpair_Encoder_model = Encoder(channel);Unpair_Decoder_model = Decoder(channel)
Unpair_Autoencoder = Model(x,Unpair_Decoder_model(Unpair_Encoder_model(x)))
Unpair_Autoencoder.load_weights(Unpair_Autoencoder_weight)
for layer in Unpair_Encoder_model.layers:layer.trainable = False
for layer in Unpair_Decoder_model.layers:layer.trainable = False
for layer in Unpair_Autoencoder.layers:layer.trainable = False

In [ ]:
K.clear_session()
image_shape = dataB[0].shape
g_model_AtoB = define_generator(image_shape, channel, resnet_n, q)
print('generator: A -> B')
g_model_BtoA = define_generator(image_shape, channel, resnet_n, q)
print('generator: B -> A')
d_model_A = define_discriminator(image_shape, q, d_lr)
print('discriminator: A -> [real/fake]')
d_model_B = define_discriminator(image_shape, q, d_lr)
print('discriminator: B -> [real/fake]')
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape, g_lr)
print('composite: A -> B -> [real/fake, A]')
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape, g_lr)
print('composite: B -> A -> [real/fake, B]')

train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA,
      Unpair_Encoder_model, Unpair_Decoder_model, dataA, dataB, model_path,
      EPA73_test_pm25, station_coordinate, extract5_list, extract10_list, extract15_list, extract20_list, ex20_KNN,
      n_epochs=epochs, n_batch=batch, save_epochs = 1, channel = channel)

print('train models done')